In [ ]:
# warning: current data partitioning with label overrides assumes one object per image

# credit to: https://towardsdatascience.com/building-your-own-object-detector-pytorch-vs-tensorflow-and-how-to-even-get-started-1d314691d4ae
# for initial ideas

# installing key packages:
# conda install pytorch torchvision torchaudio cudatoolkit=11.0 -c pytorch -c nvidia

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from PIL import Image
import random
from common import parse_kitti
import json
import numpy as np

In [ ]:
def replace_parent_path(p, parent):
    fname = os.path.basename(p)
    return os.path.join(parent, fname)

def crop(sample, pad=0):    
    image_path = sample['image_path']
    label_path = sample['label_path']
    
    img = Image.open(image_path).convert("RGB")
    
    objects = parse_kitti(label_path)                
    boxes = np.array([o['bounds'] for o in objects], dtype=int)
    boxes += np.array([-1, -1, 1 , 1], dtype=int) * pad
        
    if len(boxes) != 1:
        print(f"Warning: Crop only works for single objects per image at the moment\n{sample}")
    
    return img.crop(boxes[0])

# Data

In [ ]:
# experiment name corresponds to the foldername and file prefix for data partitions and label maps
exp_name = "t2b"
orig_image_dir = "../data/combined/images"
orig_label_dir = "../data/combined/labels"

# load train and test data partitions
test_folds = [0, 1, 2, 3, 4]

use_label_overrides = True

# Benchmark

In [ ]:
for test_fold in test_folds:

    print(f'test_fold: {test_fold}')
    train_path = f"{exp_name}/{exp_name}-train-{test_fold}.json"
    test_path = f"{exp_name}/{exp_name}-test-{test_fold}.json"

    data_train = None
    data_test = None

    with open(train_path, 'r') as f:
        data_train = json.load(f)

    with open(test_path, 'r') as f:
        data_test = json.load(f)    

    print(f"samples: {len(data_train) + len(data_test)}, train samples: {len(data_train)}, test samples: {len(data_test)}")

    # load the label map
    label_map_path = f"{exp_name}/{exp_name}-label-map.json"
    with open(label_map_path, 'r') as f:
        label_map = json.load(f)
    print(f"label map: {label_map}")
    reverse_label_map = {v:k for k,v in label_map.items()}
    
    # define datasets and data loaders
    #batch_size = 32
    #ds = KITTIDataset(data_train, label_map, use_label_overrides=use_label_overrides, transforms=get_transforms(train=False))
    #ds_test = KITTIDataset(data_test, label_map, use_label_overrides=use_label_overrides, transforms=get_transforms(train=False))

    #dl = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=1, collate_fn=utils.collate_fn)
    #dl_test = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False, num_workers=1, collate_fn=utils.collate_fn)
    
    # save cropped images
    save_dir = os.path.join(exp_name, "benchmark")
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    save_dir = os.path.join(save_dir, f"{exp_name}-part-{test_fold}")
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
        

    data_test_orig = []
    for sample in data_test:
        sample['image_path'] = replace_parent_path(sample['image_path'], orig_image_dir)
        sample['label_path'] = replace_parent_path(sample['label_path'], orig_label_dir)
        data_test_orig.append(sample)
    random.shuffle(data_test_orig)        

    answers = []
    for i, sample in enumerate(data_test_orig):        
        img_crop = crop(sample)    
        sample['test_idx'] = i
        #path = os.path.join(save_dir, os.path.basename(sample['image_path']))
        _, ext = os.path.splitext(sample['image_path'])
        path = os.path.join(save_dir, f"{sample['test_idx']}{ext}")
        img_crop.save(path)          
        answers.append(sample)

    with open(os.path.join(save_dir, f"answers.json"), 'w') as f:
        json.dump(answers, f)    

### Data Review

In [ ]:
# sample = data_test_orig[222]
# img_crop = crop(sample, pad=100)
# plt.imshow(img_crop)

In [ ]:
#plt.imshow(Image.open('../data/combined/images/25765b.jpg').convert("RGB"))